Clean Data

In [3]:
import pandas as pd
profiles = pd.read_csv('All_Profiles.csv')
devices = pd.read_csv('All_Devices.csv')
search_history = pd.read_csv('All_SearchHistory.csv')
clickstream = pd.read_csv('All_Clickstream.csv')
viewing_activity = pd.read_csv('All_ViewingActivity.csv')

PROFILES

In [5]:
profiles.shape

(9, 8)

In [6]:
profiles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Profile Name           9 non-null      object
 1   Profile Creation Time  9 non-null      object
 2   Maturity Level         9 non-null      object
 3   Primary Lang           9 non-null      object
 4   Has Auto Playback      9 non-null      object
 5   Max Stream Quality     9 non-null      object
 6   Profile Lock Enabled   9 non-null      bool  
 7   Profile Transferred    9 non-null      bool  
dtypes: bool(2), object(6)
memory usage: 582.0+ bytes


In [7]:
profiles.columns

Index(['Profile Name', 'Profile Creation Time', 'Maturity Level',
       'Primary Lang', 'Has Auto Playback', 'Max Stream Quality',
       'Profile Lock Enabled', 'Profile Transferred'],
      dtype='object')

In [38]:
# profiles = profiles.drop(['Game Handle',
#                           'Profile Transfer Time',
#                           'Profile Transferred From Account',
#                           'Profile Transferred To Account',
#                           'Date Of Birth',
#                           'Gender',
#                           'Opt-Out',
#                           'Privacy And Data Settings'], axis=1)

In [40]:
profiles['Has Auto Playback'] = profiles['Has Auto Playback'].fillna(0.0)

In [42]:
profiles['Has Auto Playback'] = profiles['Has Auto Playback'].replace({1.0: 'Yes', 0.0: 'No'})

In [44]:
profiles

,Profile Name,Profile Creation Time,Maturity Level,Primary Lang,Has Auto Playback,Max Stream Quality,Profile Lock Enabled,Profile Transferred
0,User 0,29-10-2021 05:42,ADULTS,en-IN,Yes,Unknown,False,False
1,User 1,29-10-2021 05:42,ADULTS,en-IN,Yes,Unknown,False,False
2,User 2,29-10-2021 05:42,ADULTS,en-IN,Yes,Unknown,False,False
3,User 3,29-10-2021 05:42,ADULTS,en-IN,Yes,Unknown,False,False
4,User 4,29-10-2021 05:19,ADULTS,en-IN,No,Unknown,False,False
5,User 5,07-01-2023 14:07,ADULTS,en-IN,Yes,Unknown,False,False
6,User 6,13-06-2022 18:09,ADULTS,en-IN,Yes,Unknown,False,False
7,User 7,05-06-2022 16:43,ADULTS,en-IN,Yes,Unknown,False,False
8,User 8,01-06-2022 13:24,ADULTS,en-IN,No,Unknown,False,False


In [46]:
profiles.to_csv('All_Profiles.csv', index=False)

DEVICES

In [49]:
devices.shape

(137, 12)

In [51]:
devices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 12 columns):
 #   Column                                                Non-Null Count  Dtype 
---  ------                                                --------------  ----- 
 0   Profile Name                                          137 non-null    object
 1   Esn                                                   137 non-null    object
 2   Device Type                                           137 non-null    object
 3   Acct First Playback Date                              137 non-null    object
 4   Acct Last Playback Date                               137 non-null    object
 5   Acct First Playback Date For User Generated Plays     137 non-null    object
 6   Acct Last Playback Date For User Generated Plays      137 non-null    object
 7   Profile First Playback Date                           137 non-null    object
 8   Profile Last Playback Date                            137 non-null    

In [53]:
devices.columns

Index(['Profile Name', 'Esn', 'Device Type', 'Acct First Playback Date',
       'Acct Last Playback Date',
       'Acct First Playback Date For User Generated Plays',
       'Acct Last Playback Date For User Generated Plays',
       'Profile First Playback Date', 'Profile Last Playback Date',
       'Profile First Playback Date For User Generated Plays',
       'Profile Last Playback Date For User Generated Plays',
       'Deactivation Time'],
      dtype='object')

In [55]:
print(devices.describe(include='all'))

       Profile Name    Esn    Device Type Acct First Playback Date  \
count           137    137            137                      137   
unique            9     58             12                       43   
top          User 8  Esn 0  Device Type 0               2022-07-29   
freq             42      7             43                       17   

       Acct Last Playback Date  \
count                      137   
unique                      39   
top                 2022-11-24   
freq                        17   

       Acct First Playback Date For User Generated Plays  \
count                                                137   
unique                                                42   
top                                           2022-07-26   
freq                                                  23   

       Acct Last Playback Date For User Generated Plays  \
count                                               137   
unique                                               38   


In [57]:
date_columns = devices.columns.drop(['Profile Name', 'Esn', 'Device Type'])
for column in date_columns:
    devices[column] = pd.to_datetime(devices[column], errors='coerce').dt.date
    

In [59]:
# Chuyển đổi cột ngày tháng năm thành định dạng datetime
for column in date_columns:
    devices[column] = pd.to_datetime(devices[column], errors='coerce')

# Hàm thay thế NaT bằng giá trị trung bình của cột
def fill_na_with_mean_date(series):
    valid_dates = series.dropna()
    if valid_dates.empty:
        return series.fillna(pd.to_datetime('1970-01-01'))
    
    mean_date = valid_dates.mean()
    return series.fillna(mean_date)

# Áp dụng hàm cho tất cả các cột ngày tháng năm ngoại trừ 'Deactivation Time'
for column in date_columns[:-1]:  # Exclude 'Deactivation Time'
    devices[column] = fill_na_with_mean_date(devices[column])

# Thay thế giá trị NaT trong cột 'Deactivation Time' bằng ngày mặc định 01/01/1900
devices['Deactivation Time'] = devices['Deactivation Time'].fillna(pd.to_datetime('1900-01-01'))


In [61]:
# Chuyển đổi các cột ngày tháng năm thành định dạng ngày-tháng-năm (không có thông tin về giây)
for column in date_columns:
    devices[column] = pd.to_datetime(devices[column]).dt.strftime('%Y-%m-%d')

In [63]:
devices.shape

(137, 12)

In [65]:
devices = devices.drop_duplicates()

In [67]:
devices.shape

(137, 12)

In [69]:
devices.to_csv('All_Devices.csv', index=False)

SREACH HISTORY

In [72]:
search_history.shape

(1832, 9)

In [74]:
search_history.columns

Index(['Profile Name', 'Country Iso Code', 'Device', 'Is Kids', 'Query Typed',
       'Displayed Name', 'Action', 'Section', 'Utc Timestamp'],
      dtype='object')

In [76]:
search_history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1832 entries, 0 to 1831
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Profile Name      1832 non-null   object
 1   Country Iso Code  1832 non-null   object
 2   Device            1832 non-null   object
 3   Is Kids           1832 non-null   object
 4   Query Typed       1832 non-null   object
 5   Displayed Name    1832 non-null   object
 6   Action            1832 non-null   object
 7   Section           1832 non-null   object
 8   Utc Timestamp     1832 non-null   object
dtypes: object(9)
memory usage: 128.9+ KB


In [78]:
search_history.describe(include='all')

,Profile Name,Country Iso Code,Device,Is Kids,Query Typed,Displayed Name,Action,Section,Utc Timestamp
count,1832,1832,1832,1832,1832,1832,1832,1832,1832
unique,9,5,6,1,397,813,3,3,312
top,User 8,IN,Device 0,No,No Query,Unknown,select,title_results,2023-03-18
freq,888,1433,1698,1832,465,353,1226,1348,65


In [80]:
for col in search_history.columns:
    print(search_history[col].nunique())

9
5
6
1
397
813
3
3
312


In [82]:
for col in search_history.columns[-3:-1]:
    print(search_history[col].unique())

['play' 'select' 'add']
['prequery_results' 'title_results' 'suggestion_results']


In [84]:
search_history=search_history.drop_duplicates()
search_history.shape

(1832, 9)

In [86]:

# Thay thế giá trị thiếu bằng một giá trị mặc định nếu cần
search_history['Query Typed'].fillna('No Query')
search_history['Displayed Name'].fillna('Unknown')

# Chuyển đổi cột Utc Timestamp thành kiểu datetime
search_history['Utc Timestamp'] = pd.to_datetime(search_history['Utc Timestamp'], errors='coerce').dt.date

# Loại bỏ các giá trị trùng lặp nếu cần
search_history.drop_duplicates()

search_history['Is Kids']=search_history['Is Kids'].replace({0:'No', 1:'Yes'})
search_history

,Profile Name,Country Iso Code,Device,Is Kids,Query Typed,Displayed Name,Action,Section,Utc Timestamp
0,User 4,AE,Device 0,No,No Query,"Manifest: Season 1: ""Pilot""",play,prequery_results,2022-08-01
1,User 4,AE,Device 0,No,No Query,Unknown,select,prequery_results,2022-08-01
2,User 4,AE,Device 0,No,the gray,The Gray Man,add,title_results,2022-07-17
3,User 4,AE,Device 0,No,the gray,The Gray Man,select,title_results,2022-07-17
4,User 4,AE,Device 0,No,No Query,"Stranger Things 4: ""Chapter Nine: The Piggyback""",play,prequery_results,2022-07-01
...,...,...,...,...,...,...,...,...,...
1827,User 8,IN,Device 0,No,mut,Brightburn,select,title_results,2022-06-01
1828,User 8,IN,Device 0,No,mut,TAU,select,title_results,2022-06-01
1829,User 8,IN,Device 0,No,mut,Muthu,play,title_results,2022-06-01
1830,User 8,IN,Device 0,No,mut,Muthu,select,title_results,2022-06-01


In [88]:
search_history.to_csv('All_SearchHistory.csv', index=False)

CLICKSTREAM

In [91]:
clickstream.shape

(1987, 6)

In [93]:
clickstream.columns

Index(['Profile Name', 'Source', 'Navigation Level', 'Referrer Url',
       'Webpage Url', 'Click Utc Ts'],
      dtype='object')

In [95]:
clickstream.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987 entries, 0 to 1986
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Profile Name      1987 non-null   object
 1   Source            1987 non-null   object
 2   Navigation Level  1987 non-null   object
 3   Referrer Url      1987 non-null   object
 4   Webpage Url       1987 non-null   object
 5   Click Utc Ts      1987 non-null   object
dtypes: object(6)
memory usage: 93.3+ KB


In [97]:
clickstream.describe(include='all')

,Profile Name,Source,Navigation Level,Referrer Url,Webpage Url,Click Utc Ts
count,1987,1987,1987,1987,1987,1987
unique,9,3,32,2,5,178
top,User 5,Source 0,browseTitles,Unknown,Unknown,2023-04-08
freq,582,1473,414,1985,1982,34


In [99]:
for col in clickstream.columns:
    print(clickstream[col].nunique())

9
3
32
2
5
178


In [101]:
clickstream = clickstream.drop_duplicates()

In [103]:
clickstream.shape

(1987, 6)

In [105]:
clickstream['Source'].unique()

array(['Source 0', 'Source 1', 'Source 2'], dtype=object)

In [107]:
clickstream['Click Utc Ts'] = pd.to_datetime(clickstream['Click Utc Ts'], errors='coerce').dt.date

clickstream['Referrer Url']=clickstream['Referrer Url'].fillna('Unknown')
clickstream['Webpage Url']=clickstream['Webpage Url'].fillna('Unknown')

In [109]:
clickstream

,Profile Name,Source,Navigation Level,Referrer Url,Webpage Url,Click Utc Ts
0,User 4,Source 0,signupPrompt,Unknown,Unknown,2022-10-08
1,User 4,Source 0,browseTitles,Unknown,Unknown,2022-10-08
2,User 4,Source 1,movieDetails,Unknown,Unknown,2022-09-30
3,User 4,Source 1,browseTitles,Unknown,Unknown,2022-09-30
4,User 4,Source 1,episodesSelector,Unknown,Unknown,2022-09-30
...,...,...,...,...,...,...
1982,User 8,Source 0,playback,Unknown,Unknown,2023-02-20
1983,User 8,Source 0,profilesGate,Unknown,Unknown,2023-02-20
1984,User 8,Source 0,comedyFeedTab,Unknown,Unknown,2023-02-20
1985,User 8,Source 0,search,Unknown,Unknown,2023-02-20


In [111]:
clickstream=clickstream.drop_duplicates()
clickstream

,Profile Name,Source,Navigation Level,Referrer Url,Webpage Url,Click Utc Ts
0,User 4,Source 0,signupPrompt,Unknown,Unknown,2022-10-08
1,User 4,Source 0,browseTitles,Unknown,Unknown,2022-10-08
2,User 4,Source 1,movieDetails,Unknown,Unknown,2022-09-30
3,User 4,Source 1,browseTitles,Unknown,Unknown,2022-09-30
4,User 4,Source 1,episodesSelector,Unknown,Unknown,2022-09-30
...,...,...,...,...,...,...
1982,User 8,Source 0,playback,Unknown,Unknown,2023-02-20
1983,User 8,Source 0,profilesGate,Unknown,Unknown,2023-02-20
1984,User 8,Source 0,comedyFeedTab,Unknown,Unknown,2023-02-20
1985,User 8,Source 0,search,Unknown,Unknown,2023-02-20


In [113]:
clickstream.to_csv('All_Clickstream.csv', index=False)

VIEWING ACTIVITY

In [116]:
viewing_activity.shape

(9992, 9)

In [118]:
viewing_activity.columns

Index(['Profile Name', 'Start Time', 'Duration', 'Title', 'Device Type',
       'Bookmark', 'Latest Bookmark', 'Country', 'End Time'],
      dtype='object')

In [120]:
viewing_activity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9992 entries, 0 to 9991
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Profile Name     9992 non-null   object
 1   Start Time       9992 non-null   object
 2   Duration         9992 non-null   object
 3   Title            9992 non-null   object
 4   Device Type      9992 non-null   object
 5   Bookmark         9992 non-null   object
 6   Latest Bookmark  9992 non-null   object
 7   Country          9992 non-null   object
 8   End Time         9992 non-null   object
dtypes: object(9)
memory usage: 702.7+ KB


In [122]:
viewing_activity.describe(include='all')

,Profile Name,Start Time,Duration,Title,Device Type,Bookmark,Latest Bookmark,Country,End Time
count,9992,9992,9992,9992,9992,9992,9992,9992,9992
unique,9,9988,2741,4327,9,3532,1791,10,9991
top,User 4,2022-08-03 19:31:35,0 days 00:00:04,Minnal Murali,Device Type 0,00:00:04,Not latest view,IN (INDIA),2022-11-25 16:55:26
freq,5239,2,320,40,4966,256,4828,5796,2


In [124]:
for col in viewing_activity.columns:
    print(viewing_activity[col].nunique())

9
9988
2741
4327
9
3532
1791
10
9991


In [126]:
viewing_activity=viewing_activity.drop_duplicates()

In [128]:
viewing_activity

,Profile Name,Start Time,Duration,Title,Device Type,Bookmark,Latest Bookmark,Country,End Time
0,User 1,2022-04-28 16:08:56,0 days 00:00:06,Chernobyl 1986,Device Type 1,01:02:16,01:02:16,IN (INDIA),2022-04-28 16:09:02
1,User 1,2022-04-28 15:54:30,0 days 00:09:39,13 Hours: The Secret Soldiers of Benghazi,Device Type 1,02:13:19,02:13:19,IN (INDIA),2022-04-28 16:04:09
2,User 1,2022-04-28 15:53:20,0 days 00:00:06,Saving Private Ryan,Device Type 1,01:16:17,01:16:17,IN (INDIA),2022-04-28 15:53:26
3,User 1,2022-04-28 15:52:18,0 days 00:00:15,Mosul,Device Type 1,01:06:44,01:06:44,IN (INDIA),2022-04-28 15:52:33
4,User 1,2022-04-28 15:49:11,0 days 00:01:45,12 Strong,Device Type 1,00:52:28,00:52:28,IN (INDIA),2022-04-28 15:50:56
...,...,...,...,...,...,...,...,...,...
9987,User 5,2023-01-07 18:41:03,0 days 00:00:26,Slumberland - Fast Laughs - Clip 3,Device Type 1,00:00:26,00:00:26,IN (INDIA),2023-01-07 18:41:29
9988,User 5,2023-01-07 15:44:54,0 days 00:02:14,Wednesday: Season 1: Wednesday's Child Is Full...,Device Type 5,00:23:25,Not latest view,IN (INDIA),2023-01-07 15:47:08
9989,User 5,2023-01-07 15:13:36,0 days 00:21:11,Wednesday: Season 1: Wednesday's Child Is Full...,Device Type 5,00:21:11,Not latest view,IN (INDIA),2023-01-07 15:34:47
9990,User 5,2023-01-07 15:13:26,0 days 00:00:06,Season 1 Genre Specific Moment Clip: Wednesday,Device Type 5,00:00:06,Not latest view,IN (INDIA),2023-01-07 15:13:32


In [130]:
viewing_activity['Attributes'].unique()

KeyError: 'Attributes'

In [ ]:
viewing_activity['Supplemental Video Type'].unique()

In [ ]:
df2 = viewing_activity[viewing_activity['Supplemental Video Type'].isnull()]

In [ ]:
df2

In [ ]:
# Chuyển đổi cột Start Time thành datetime
viewing_activity['Start Time'] = pd.to_datetime(viewing_activity['Start Time'])

# Chuyển đổi cột Duration thành timedelta
def parse_duration(duration_str):
    """Chuyển đổi chuỗi thời gian sang timedelta."""
    if pd.isna(duration_str):
        return pd.Timedelta(0)
    hours, minutes, seconds = map(int, duration_str.split(':'))
    return pd.Timedelta(hours=hours, minutes=minutes, seconds=seconds)

viewing_activity['Duration'] = viewing_activity['Duration'].apply(parse_duration)


viewing_activity['Country'] = viewing_activity['Country'].str.upper().str.strip()

viewing_activity['End Time'] = viewing_activity['Start Time'] + viewing_activity['Duration']

viewing_activity.drop(['Attributes', 'Supplemental Video Type'], axis=1, inplace=True)



In [ ]:
viewing_activity.to_csv('All_ViewingActivity.csv', index=False)

FINISHED CLEANING DATA